In [104]:
import pymongo
import numpy as np

#Connect to mongoDB and get data from collection
client = pymongo.MongoClient() 
db = client.TrainingDataDB 

currentExpData = []
discExpData = []
for i in range(1,33):
    expCollectionName = "TakeOffExp"+str(i)
    
    expCollection = db[expCollectionName]
    
    expDocuments = expCollection.find({})
    currentTrialData = []
    for expDocument in expDocuments:
        expDataArray = expDocument['data']
        currentExpData.append(expDataArray)
        #For breaking the data into smaller pieces
        currentTrialData.append(expDataArray)
    currTrial = np.array(currentTrialData) #make it into numpy array
    discCurrTrial = np.array([currTrial[x:x+30] for x in range(0, len(currTrial), 30)])
    #print discCurrTrial
    #print "-------------------"
    #print currTrial.shape
    #print len(discCurrTrial)
    discExpData.append(discCurrTrial)
    #print discExpData

#print currentExpData
#print discExpData

expertDataArray = np.array(currentExpData)
discExpDataArray = np.array(discExpData)
print expertDataArray.shape
#print discExpDataArray

(3390L, 10L)


In [105]:
currentNovData = []      
for j in range (1,57):
    novCollectionName = "TakeOffNov"+str(j)
    
    novCollection = db[novCollectionName]
    
    novDocuments = novCollection.find({})
    if j==3:
        print "Skipping Trial #3" #Due to null values from the sensor which cause the classifier to crash
    else:
        for novDocument in novDocuments:
            novDataArray = novDocument['data']
            currentNovData.append(novDataArray)
            currTrial = np.array(currentNovData)
            #print currTrial.shape

#print currentNovData

noviceDataArray = np.array(currentNovData)
print noviceDataArray.shape

Skipping Trial #3
(4695L, 10L)


In [106]:
#Consolidate all data 
allData = np.concatenate((expertDataArray,noviceDataArray), axis=0)
allData.shape

(8085L, 10L)

In [107]:
#Create labels
expert_rows, expert_columns = expertDataArray.shape
#print expert_rows
expert_labels = np.zeros(expert_rows,)
#print expert_labels
novice_rows, novice_columns = noviceDataArray.shape
#print novice_rows
novice_labels = np.ones(novice_rows,)
#print novice_labels

all_labels = np.concatenate((expert_labels,novice_labels), axis=0)
print all_labels

[ 0.  0.  0. ...,  1.  1.  1.]


In [108]:
from sklearn import preprocessing, cross_validation, neighbors, svm, tree
Data_train, Data_test, Labels_train, Labels_test = cross_validation.train_test_split(allData, all_labels, test_size=0.2)

import time

#KNN
knn_classifier = neighbors.KNeighborsClassifier()
start_time = time.time()
knn_classifier.fit(Data_train, Labels_train)
#knn_classifier.fit(Data_test, Labels_test)
knn_score = knn_classifier.score(Data_test, Labels_test)
stop_time = time.time()
knn_runtime = stop_time-start_time
print knn_score
print knn_runtime

0.835497835498
0.0269999504089


In [109]:
#SVM
svm_classifier = svm.SVC(kernel='linear', C = 1.0)
start_time = time.time()
svm_classifier.fit(Data_train, Labels_train)
#svm_classifier.fit(Data_test, Labels_test)
svm_score = svm_classifier.score(Data_test, Labels_test)
stop_time = time.time()
svm_runtime = stop_time-start_time
print svm_score
print svm_runtime

0.810760667904
21.9049999714


In [110]:
#DTree
dt_classifier = tree.DecisionTreeClassifier()
start_time = time.time()
dt_classifier.fit(Data_train, Labels_train)
#dt_classifier.fit(Data_test, Labels_test)
dtree_score = dt_classifier.score(Data_test, Labels_test)
stop_time = time.time()
dt_runtime = stop_time-start_time
print dtree_score
print dt_runtime

0.805194805195
0.0440001487732


In [111]:
collection_name = "TakeOffNov61"
collection = db[collection_name]
documents = collection.find({})
currentTrialData = []
for document in documents:
    dataArray = document['data']
    currentTrialData.append(dataArray)

dataArray = np.array(currentTrialData)
dataArray.shape


(108L, 10L)

In [112]:
#predict using KNN
knn_result = knn_classifier.predict(dataArray)
#print result

length = len(knn_result)
#print length
num_ones = np.count_nonzero(knn_result,axis=0)
#print num_ones
if (num_ones/float(length)) > 0.50:
    print "Novice"
else:
    print "Expert"
print num_ones/float(length)

Novice
0.888888888889


In [113]:
#predict using SVM
svm_result = svm_classifier.predict(dataArray)
length = len(svm_result)
#print length
num_ones = np.count_nonzero(svm_result,axis=0)
#print num_ones
if (num_ones/float(length)) > 0.50:
    print "Novice"
else:
    print "Expert"
print num_ones/float(length)

Novice
0.981481481481


In [114]:
#predict using Decision Tree
dt_result = dt_classifier.predict(dataArray)
length = len(dt_result)
#print length
num_ones = np.count_nonzero(dt_result,axis=0)
#print num_ones
if (num_ones/float(length)) > 0.50:
    print "Novice"
else:
    print "Expert"
print num_ones/float(length)

Novice
0.759259259259
